In [1]:
import gymnasium as gym
import numpy as np
from random import sample
import pandas as pd
from utilities import split_in_blocks, float_to_bits, int_to_bits
from constants import parameters_dict
import math

class CompressionEnv(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.Box(low=np.array([0, 0, 0, 0, 0, 0, 0, 0, 0]), high=np.array([5, 5, 5, 5, 5, 7, 1, 1, 1]), shape=(9,),  dtype=np.float64)
        self.observation_space = gym.spaces.Box(low=np.array([0 for i in range(34 * 8)] ), high=np.array([255 for i in range(34 * 8)]), shape=(34 * 8,),  dtype=np.float32)
        self.idx = 0
        self.timeseries = ['Electric_Production.csv', 'Gold.csv', 'Electric_Production.csv', 'daily-temperatures.csv', 'oil.csv', 'transactions.csv']
        self.timestamps = []
        self.metrics = []
        self.crt_vec = 'timestamps'
        self.paramters = parameters_dict
        self.last_2_time = []
        self.last_2_metric = []
        self.get_ts()
        

    def get_ts(self):
        my_ts = sample(self.timeseries, 1)
        df = pd.read_csv(my_ts[0])

        try:
            #Convert the date to timestamp
            self.timestamps = pd.to_datetime(df['DATE'], format='%Y-%m-%d').astype(np.int64).values // 1000000
        except:
            try:
                self.timestamps = pd.to_datetime(df['DATE'], format='%d-%m-%Y').astype(np.int64).values // 1000000
            except:
                try:
                    self.timestamps = pd.to_datetime(df['DATE'], format='%m-%d-%Y').astype(np.int64).values // 1000000
                except:
                    try:
                        self.timestamps = pd.to_datetime(df['DATE'], format='%Y/%d/%m').astype(np.int64).values // 1000000
                    except:
                        try:
                            self.timestamps = pd.to_datetime(df['DATE'], format='%d/%m/%Y').astype(np.int64).values // 1000000
                        except:
                            self.timestamps = pd.to_datetime(df['DATE'], format='%m/%d/%Y').astype(np.int64).values // 1000000
        self.metrics = (df.iloc[:, 1]).astype(np.float64).values
        self.metrics[np.isnan(self.metrics)==True] = 0
        self.last_2_time = self.timestamps[:2]
        self.last_2_metric = self.metrics[:2]
        self.timestamps = split_in_blocks(self.timestamps[2:])
        self.metrics = split_in_blocks(self.metrics[2:])

        self.state = self.timestamps[0]

    def bytes(self, vec):
        res = []
        for x in vec:
            if self.crt_vec == 'timestamps':
                y = int_to_bits(x)
            else:
                y = float_to_bits(x)
            for start in range(0, 64, 8):
                byte = y[start:start+8]
                byte = int(byte, 2)
                res.append(byte)
        return np.asarray(res).astype(np.float32)

    def step(self, action):
        params = np.round(action).astype(int)
        bits_count = self.compress(params)

        reward = self.evaluate(self.state, bits_count)
        self.idx += 1
        if self.idx >= len(self.timestamps) and self.crt_vec == 'timestamps':
            self.idx = 0
            self.crt_vec = 'metrics'
        elif self.idx >= len(self.metrics) and self.crt_vec == 'metrics':
            state = np.pad(self.state, (0, 32 - len(self.state)), 'constant', constant_values=(-1e14, ))
            return self.bytes(np.concatenate([self.last_2_time, state])), reward, True, False, {}

        if self.crt_vec == 'timestamps':
            self.state = self.timestamps[self.idx]
        else:
            self.state = self.metrics[self.idx]

        if self.crt_vec == 'timestamps':
            if len(self.state) == 32:
                return self.bytes(np.concatenate([self.last_2_time, self.state])), reward, False, False, {}
            state = np.pad(self.state, (0, 32 - len(self.state)), 'constant', constant_values=(-1e14, ))
            return self.bytes(np.concatenate([self.last_2_time, state])), reward, False, False, {}
        else:
            if len(self.state) == 32:
                return self.bytes(np.concatenate([self.last_2_metric, self.state])), reward, False, False, {}
            state = np.pad(self.state, (0, 32 - len(self.state)), 'constant', constant_values=(-1e14, ))
            return self.bytes(np.concatenate([self.last_2_metric, state])), reward, False, False, {}
        
    def reset(self, seed=None):
        self.get_ts()
        self.crt_vec = 'timestamps'
        return self.bytes(np.concatenate([self.last_2_time, self.state])), {}

    def evaluate(self, original, compressed):
        val = (compressed / (len(original) * 64))
        return -math.tan(val) / 1.2 + 0.6

    def compress(self, params):
        transTypes = [self.paramters[1][params[0]], self.paramters[2][params[1]]]
        transCodings = ['0', '1']
        compressTypes = [self.paramters[4][params[2]], self.paramters[5][params[3]], self.paramters[6][params[4]]]
        compressCodings = ['00', '01', '11']
        offByteShift1 = params[5]
        offByteShift2 = params[6]
        offByteShift3 = params[7]
        offBitmask = params[8] 
        #endian = params[10]

        sum = 0

        for i in range(0, len(self.state) - 1, 2):

            first, second = self.state[i], self.state[i + 1]
            opt1 = 64
            opt2 = 64

            if self.crt_vec == 'timestamps':
                last_2 = self.last_2_time
            else:
                last_2 = self.last_2_metric
            val1 = val2 = 0
            for transformer in transTypes:
                if self.crt_vec == 'metrics':
                    if transformer.__name__ == 'xor' or transformer.__name__ == 'delta_xor':
                        if self.crt_vec == 'timestamps':
                            self.last_2_time = [self.state[i], self.state[i + 1]]
                        else:
                            self.last_2_metric = [self.state[i], self.state[i + 1]]
                        continue
                    val1 = transformer(last_2[0], last_2[1], first)
                    val2 = transformer(last_2[1], first, second)
                else:
                    val1 = transformer(last_2[0], last_2[1], first)
                    val2 = transformer(last_2[1], first, second)

                if self.crt_vec == 'timestamps':
                    val1 = int_to_bits(val1)
                    val2 = int_to_bits(val2)
                else:
                    val1 = float_to_bits(val1)
                    val2 = float_to_bits(val2)

                for (compresser, param) in  compressTypes:
                    if compresser.__name__ == 'bitmask' and param == 2:
                        if self.crt_vec == 'timestamps':
                            self.last_2_time = [self.state[i], self.state[i + 1]]
                        else:
                            self.last_2_metric = [self.state[i], self.state[i + 1]]
                        continue
                    else:
                        if compresser.__name__ == 'offset':
                            match param:
                                case 1:
                                    opt1 = min(opt1, compresser(val1, offByteShift1))
                                    opt2 = min(opt2, compresser(val2, offByteShift1))
                                case 2:
                                    if offByteShift1 - offByteShift2 >= 0:
                                        opt1 = min(opt1, compresser(val1, offByteShift1 - offByteShift2))
                                        opt2 = min(opt2, compresser(val2, offByteShift1 - offByteShift2))
                                case 3:
                                    if offByteShift1 - offByteShift2 - offByteShift3 >= 0:
                                        opt1 = min(opt1, compresser(val1, offByteShift1 - offByteShift2 - offByteShift3))
                                        opt2 = min(opt2, compresser(val2, offByteShift1 - offByteShift2 - offByteShift3))
                        elif compresser.__name__ == 'bitmask':
                            opt1 = min(opt1, compresser(val1, offBitmask))
                            opt2 = min(opt1, compresser(val2, offBitmask))
                        else:
                            opt1 = min(opt1, compresser(val1))
                            opt2 = min(opt1, compresser(val2))
            if self.crt_vec == 'timestamps':
                self.last_2_time = [self.state[i], self.state[i + 1]]
            else:
                self.last_2_metric = [self.state[i], self.state[i + 1]]
            sum += opt1 + opt2
        return sum


    def render(self):
        #useless
        pass

    def close(self):
        #useless
        pass

In [2]:
env = CompressionEnv()
action = env.action_space.sample()
state, reward, done, truncated, info = env.step(action)
sum = reward
ct = 1
maxi = 0
while not done:
    action = env.action_space.sample()
    state, reward, done, truncarted, info = env.step(action)
    #print(reward, action, state)
    maxi = max(maxi, reward)
    sum += reward
    ct += 1
print(sum / ct, maxi, ct)

0.4326987163284577 0.5869781069250468 5218


In [3]:
from stable_baselines3.common.env_checker import check_env
env = CompressionEnv()
obs, _ = env.reset()
print(obs)
check_env(env)

2024-05-24 16:39:04.121532: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 16:39:04.121582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 16:39:04.122496: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-24 16:39:04.129695: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 16:39:05.100604: W tensorflow/compiler/tf2

[  0.   0.   0. 183. 185. 128.  44.   0.   0.   0.   0. 183. 190. 166.
 136.   0.   0.   0.   0. 183. 195. 204. 228.   0.   0.   0.   0. 183.
 200. 243.  64.   0.   0.   0.   0. 183. 216. 102.  84.   0.   0.   0.
   0. 183. 221. 140. 176.   0.   0.   0.   0. 183. 226. 179.  12.   0.
   0.   0.   0. 183. 231. 217. 104.   0.   0.   0.   0. 183. 236. 255.
 196.   0.   0.   0.   0. 183. 252. 114. 216.   0.   0.   0.   0. 184.
   1. 153.  52.   0.   0.   0.   0. 184.   6. 191. 144.   0.   0.   0.
   0. 184.  11. 229. 236.   0.   0.   0.   0. 184.  17.  12.  72.   0.
   0.   0.   0. 184.  32. 127.  92.   0.   0.   0.   0. 184.  37. 165.
 184.   0.   0.   0.   0. 184.  42. 204.  20.   0.   0.   0.   0. 184.
  47. 242. 112.   0.   0.   0.   0. 184.  53.  24. 204.   0.   0.   0.
   0. 184.  68. 139. 224.   0.   0.   0.   0. 184.  73. 178.  60.   0.
   0.   0.   0. 184.  78. 216. 152.   0.   0.   0.   0. 184.  83. 254.
 244.   0.   0.   0.   0. 184.  89.  37.  80.   0.   0.   0.   0. 184.
 104. 

/home/dimi/.local/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
/home/dimi/.local/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:464: UserWarning: Your action space has dtype float64, we recommend using np.float32 to avoid cast errors.
  warnings.warn(


In [4]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [5]:
#Use SAC from stable baselines
from stable_baselines3 import TD3
from stable_baselines3.sac.policies import MlpPolicy, MultiInputPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

env = CompressionEnv()
env = DummyVecEnv([lambda: env])

n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

#Add progress bar for learning
model = TD3('MlpPolicy', env, action_noise=action_noise, verbose=1)
model3 = model.learn(total_timesteps=50000, progress_bar=True)


Using cuda device


Output()

---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 58       |
|    time_elapsed    | 91       |
|    total_timesteps | 5409     |
| train/             |          |
|    actor_loss      | 1.97     |
|    critic_loss     | 0.355    |
|    learning_rate   | 0.001    |
|    n_updates       | 5308     |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 59       |
|    time_elapsed    | 97       |
|    total_timesteps | 5808     |
| train/             |          |
|    actor_loss      | 2.09     |
|    critic_loss     | 0.488    |
|    learning_rate   | 0.001    |
|    n_updates       | 5707     |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 12       |
|    fps             | 57       |
|    time_elapsed    | 113      |
|    total_timesteps | 6544     |
| train/             |          |
|    actor_loss      | 2.16     |
|    critic_loss     | 0.337    |
|    learning_rate   | 0.001    |
|    n_updates       | 6443     |
---------------------------------


...

---------------------------------
| time/              |          |
|    episodes        | 68       |
|    fps             | 61       |
|    time_elapsed    | 480      |
|    total_timesteps | 29742    |
| train/             |          |
|    actor_loss      | 10.9     |
|    critic_loss     | 0.911    |
|    learning_rate   | 0.001    |
|    n_updates       | 29641    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 72       |
|    fps             | 62       |
|    time_elapsed    | 562      |
|    total_timesteps | 35013    |
| train/             |          |
|    actor_loss      | 10.1     |
|    critic_loss     | 0.618    |
|    learning_rate   | 0.001    |
|    n_updates       | 34912    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 76       |
|    fps             | 62       |
|    time_elapsed    | 645      |
|    total_timesteps | 40461    |
| train/             |          |
|    actor_loss      | 10.7     |
|    critic_loss     | 0.477    |
|    learning_rate   | 0.001    |
|    n_updates       | 40360    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 80       |
|    fps             | 62       |
|    time_elapsed    | 647      |
|    total_timesteps | 40616    |
| train/             |          |
|    actor_loss      | 10.4     |
|    critic_loss     | 1.1      |
|    learning_rate   | 0.001    |
|    n_updates       | 40515    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 84       |
|    fps             | 63       |
|    time_elapsed    | 727      |
|    total_timesteps | 46258    |
| train/             |          |
|    actor_loss      | 11.4     |
|    critic_loss     | 0.572    |
|    learning_rate   | 0.001    |
|    n_updates       | 46157    |
---------------------------------


In [31]:
model3 = model.learn(total_timesteps=10000, progress_bar=True)


Output()

---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 35       |
|    time_elapsed    | 7        |
|    total_timesteps | 265      |
| train/             |          |
|    actor_loss      | -3.02    |
|    critic_loss     | 0.179    |
|    learning_rate   | 0.001    |
|    n_updates       | 10064    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 23       |
|    time_elapsed    | 28       |
|    total_timesteps | 689      |
| train/             |          |
|    actor_loss      | -4.22    |
|    critic_loss     | 0.163    |
|    learning_rate   | 0.001    |
|    n_updates       | 10488    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 12       |
|    fps             | 19       |
|    time_elapsed    | 54       |
|    total_timesteps | 1064     |
| train/             |          |
|    actor_loss      | -3.99    |
|    critic_loss     | 0.151    |
|    learning_rate   | 0.001    |
|    n_updates       | 10863    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 16       |
|    fps             | 17       |
|    time_elapsed    | 109      |
|    total_timesteps | 1859     |
| train/             |          |
|    actor_loss      | -4.87    |
|    critic_loss     | 1.39     |
|    learning_rate   | 0.001    |
|    n_updates       | 11658    |
---------------------------------


---------------------------------
| time/              |          |
|    episodes        | 20       |
|    fps             | 16       |
|    time_elapsed    | 138      |
|    total_timesteps | 2335     |
| train/             |          |
|    actor_loss      | -5.9     |
|    critic_loss     | 0.299    |
|    learning_rate   | 0.001    |
|    n_updates       | 12134    |
---------------------------------


In [6]:
#save the model
model.save("TD3_CompressionEnv")

In [14]:
model2 = TD3('MlpPolicy', env, action_noise=action_noise, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [12]:
env = CompressionEnv()
obs, _ = env.reset()

action, _states = model.predict(obs)
state, reward, done, truncated, info = env.step(action)
sum = reward
ct = 1
while not done:
    action, _state = model3.predict(state)
    state, reward, done, truncarted, info = env.step(action)
    sum += reward
    ct += 1
print(reward / ct)
print(reward, ct)
print('ceva')

-0.00013373702003559833
-0.6978397705457521 5218
ceva


In [ ]:
from transform_primitives import xor, delta_xor, delta, delta_of_delta, rev_delta, rev_delta_of_delta
from compression_primitives import bitmask, offset, trailing_zero
env = CompressionEnv()
#env.get_ts()
print(env.metrics[0])
sample = env.metrics[0]
prev = env.last_2_metric
print(prev)
transformers = [delta, delta_of_delta, rev_delta, rev_delta_of_delta]
avg = 0
ct = 0
for j in range(len(env.metrics)):
    sample = env.metrics[j]
    for i in range(len(sample)):
        ct += 1
        mini = 64
        for t in transformers:
            val = t(prev[0], prev[1], sample[i])
            val = int(num_to_bits(val), 2)
            mini = min(mini, bitmask(val, 0), offset(val, 0), trailing_zero(val), offset(val, 1), offset(val, 2), bitmask(val, 1))
        avg += mini
        prev[0] = prev[1]
        prev[1] = sample[i]
print(avg / ct)
#daily-temperature 61.26 avg
#gold 61.82

In [ ]:
action = [3, 0, 1, 3, 0, 1, 0, 0, 0, 0]
env2 = CompressionEnv()
print(env2.state)
state, reward, done, truncated, info = env2.step(action)
print(reward)

In [ ]:
df = pd.read_csv('Gold.csv')
x = pd.to_datetime(df['DATE'], format='%Y/%m/%d')
print(x.astype(np.int64))

In [ ]:
from utilities import int_to_bits, float_to_bits
print(int_to_bits(57), float_to_bits(2.257))